In [4]:
#For the dataset used is vgsales
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# 1. Load Data
df = pd.read_csv('vgsales.csv')

# 2. Basic Cleanup
df['Publisher'] = df['Publisher'].fillna('Unknown')

# 3. LABEL ENCODING (Essential for KNN)
# We convert text columns to numbers so KNN can understand them.
le_genre = LabelEncoder()
le_platform = LabelEncoder()
le_publisher = LabelEncoder()

df['Genre_Code'] = le_genre.fit_transform(df['Genre'])
df['Platform_Code'] = le_platform.fit_transform(df['Platform'])
df['Publisher_Code'] = le_publisher.fit_transform(df['Publisher'])

# 4. KNN IMPUTATION
# Select features to help predict the missing Year
features = ['Platform_Code', 'Genre_Code', 'Publisher_Code',
            'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Year']

# Scale the data (0-1) so large sales numbers don't dominate the distance calc
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[features]), columns=features)

# Run KNN to fill missing values
imputer = KNNImputer(n_neighbors=5)
df_imputed_scaled = pd.DataFrame(imputer.fit_transform(df_scaled), columns=features)

# Inverse scale to get back the real Year values
df_final = scaler.inverse_transform(df_imputed_scaled)
df_final = pd.DataFrame(df_final, columns=features)

# 5. Finalize Dataset
# Update the original Year column with the imputed values
df['Year'] = df_final['Year'].round().astype(int)

# Create a final clean dataframe with ONLY the numeric columns (Label Encoded)
df_ml_ready = df[['Year', 'Genre_Code', 'Platform_Code', 'Publisher_Code',
                  'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']]

print("Success! Final Data Shape:", df_ml_ready.shape)
display(df_ml_ready.head())

# Save
df_ml_ready.to_csv('vgsales_final_label_knn.csv', index=False)

# FEATURE ENGINEERING: Creating 'Sales_Tier'
# We use np.select for efficient multi-condition categorization
conditions = [
    (df['Global_Sales'] >= 1.0),                  # Blockbuster
    (df['Global_Sales'] >= 0.1) & (df['Global_Sales'] < 1.0), # Moderate
    (df['Global_Sales'] < 0.1)                    # Niche
]

# The labels for each condition
tier_labels = ['Blockbuster', 'Moderate', 'Niche']

# Create the new column
df['Sales_Tier'] = np.select(conditions, tier_labels, default='Unknown')

# Check the result
print("--- Tier Distribution ---")
print(df['Sales_Tier'].value_counts())

print("\n--- Top 5 Rows with Tiers ---")
display(df[['Name', 'Global_Sales', 'Sales_Tier']].head())

# Save
df.to_csv('vgsales_feature_engineered.csv', index=False)

Success! Final Data Shape: (16598, 9)


,Year,Genre_Code,Platform_Code,Publisher_Code,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,2006,10,26,359,41.49,29.02,3.77,8.46,82.74
1,1985,4,11,359,29.08,3.58,6.81,0.77,40.24
2,2008,6,26,359,15.85,12.88,3.79,3.31,35.82
3,2009,10,26,359,15.75,11.01,3.28,2.96,33.00
4,1996,7,5,359,11.27,8.89,10.22,1.00,31.37


--- Tier Distribution ---
Sales_Tier
Moderate       8736
Niche          5781
Blockbuster    2081
Name: count, dtype: int64

--- Top 5 Rows with Tiers ---


,Name,Global_Sales,Sales_Tier
0,Wii Sports,82.74,Blockbuster
1,Super Mario Bros.,40.24,Blockbuster
2,Mario Kart Wii,35.82,Blockbuster
3,Wii Sports Resort,33.00,Blockbuster
4,Pokemon Red/Pokemon Blue,31.37,Blockbuster


In [5]:
#This we used the dataset from the champions leagues matches
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

# 1. Load Data
df = pd.read_csv('champions_league_matches.csv')
df = df.dropna(subset=['score']).copy()

# Helper: Parse Score
def parse_score(score_str):
    if pd.isna(score_str): return np.nan, np.nan
    clean_score = str(score_str).replace('–', '-').replace('—', '-')
    if '-' in clean_score:
        parts = clean_score.split('-')
        return int(parts[0]), int(parts[1])
    return np.nan, np.nan

df[['home_goals', 'away_goals']] = df['score'].apply(lambda x: pd.Series(parse_score(x)))


# Data Preprocessing


# A. Label Encoding (IDs)
le = LabelEncoder()
all_teams = pd.concat([df['home_team'], df['away_team']]).unique()
le.fit(all_teams)
df['Home_Team_ID'] = le.transform(df['home_team'])
df['Away_Team_ID'] = le.transform(df['away_team'])

# B. Target Encoding (Strength)
# We encode teams based on how many goals they usually score
home_strength = df.groupby('home_team')['home_goals'].transform('mean')
away_strength = df.groupby('away_team')['away_goals'].transform('mean')
df['Home_Strength_Enc'] = home_strength
df['Away_Strength_Enc'] = away_strength

# C. KNN Imputation (Filling Gaps)
# Clean possession to float first
df['home_possession'] = df['home_possession'].astype(str).str.replace('%', '').astype(float)
# Impute missing Save % using Possession and Team ID
impute_cols = ['Home_Team_ID', 'home_possession', 'home_saves_pct']
imputer = KNNImputer(n_neighbors=3)
df_filled = pd.DataFrame(imputer.fit_transform(df[impute_cols]), columns=impute_cols)
df['home_saves_pct_imputed'] = df_filled['home_saves_pct']

# D. Cyclical Encoding (Time)
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

# FEATURE ENGINEERING

# 1. Relative Strength (Interaction of Target Encodings)
# Positive = Home is stronger. Negative = Away is stronger.
df['Relative_Strength'] = df['Home_Strength_Enc'] - df['Away_Strength_Enc']

# 2. Defensive Vulnerability (Interaction of KNN Imputed Data)
# High Score = Team allows lots of possession and saves few shots
df['Defensive_Vulnerability'] = (100 - df['home_possession']) * (100 - df['home_saves_pct_imputed']) / 100

# 3. Match Potential (Sum of Target Encodings)
# High Score = Two heavy hitters playing each other
df['Match_Quality'] = df['Home_Strength_Enc'] + df['Away_Strength_Enc']

# View the final "Super Features"
print("--- Final Engineered Features ---")
display(df[['home_team', 'away_team', 'Relative_Strength', 'Defensive_Vulnerability', 'Match_Quality']].head())

# Save
df.to_csv('champions_league_full_features.csv', index=False)

--- Final Engineered Features ---


,home_team,away_team,Relative_Strength,Defensive_Vulnerability,Match_Quality
0,PSV Eindhoven,Union SG,1.333333,12.654,4.666667
1,Athletic Club,Arsenal,-1.666667,20.313,3.666667
2,Tottenham,Villarreal,2.666667,4.662,2.666667
3,Benfica,Qarabağ,0.000000,28.200,2.666667
4,Juventus,Dortmund,-0.666667,36.000,5.333333
